# Make Marine Resistance and Hubs

Created by Amy Keister, last run by Amy Keister on September 7 2023. It took 50 minutes to run.

In [1]:
import os
import arcpy
import numpy

In [2]:
import time
start = time.time()

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace = 
OutWorkspace = r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\E_ConMarineResistanceAndHubs.gdb"

In [4]:
# define rasters used for cell size, extent, and snapping
Rextent= r"D:\SE_Blueprint_2023\2_FullExtent\SEBlueprint2023ExtentVIPR6.tif"

In [45]:
# define final outputs
Hubs= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ea_ConMarine_LM_Input.gdb\Hubs"
Resist= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ea_ConMarine_LM_Input.gdb\Resistance"

HubsGoM= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ea_ConMarine_LM_Input.gdb\HubsGoM"
ResistGoM= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ea_ConMarine_LM_Input.gdb\ResistanceGoM"

HubsA= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ea_ConMarine_LM_Input.gdb\HubsA"
ResistA= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ea_ConMarine_LM_Input.gdb\ResistanceA"

In [6]:
# define additional inputs
subR = r"D:\SE_Blueprint_2023\2_FullExtent\SEBlueprint2023SubregionsVIPR6.shp"
estuary= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\A_ConInlandResistance.gdb\estuaryDeep"
Inland= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\A_ConInlandResistance.gdb\InlandNearShoreBuffRNoEst"
Zonation= r"D:\SE_Blueprint_2023\7_CombineZonation\CombineZonation.tif"
# this was created using the E_MarineMammalAndTurtleMovementAreas.ipynb
mask= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\MarineDataFromRua\MarineMammalAndTurtleMovementAreas\MarineMammalAndTurtleMovementAreas.tif"

# Start Analysis

In [7]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutWorkspace

In [8]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\E_ConMarineResistanceAndHubs.gdb


# Define Continental estuarine/marine area
I'm using the inland definition created for the inland resistance layer and just picking everything else in the Continental extent.
This looks unnecessarily complicated but it works

In [9]:
# pull out only the Continental polygons from the subregions
arcpy.analysis.Select(subR, "NotCaribbean", "SubRgn <> 'Caribbean'")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\E_ConMarineResistanceAndHubs.gdb\\NotCaribbean'>

In [10]:
# reclassify inland
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="NotCaribbean", snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Reclassify(Inland, "Value", "1 1;NODATA 0", "DATA"); out_raster.save("Inland1")

In [11]:
# reduce area to estuarine/marine extent
out_raster = arcpy.sa.Times(Rextent, "Inland1"); out_raster.save("EmarineCar")

In [12]:
# remove Caribbean
out_raster = arcpy.sa.ExtractByMask(
    in_raster="EmarineCar",
    in_mask_data="NotCaribbean",
    extraction_area="INSIDE",
    analysis_extent= Rextent
)
out_raster.save("Emarine")

In [13]:
# reclassify to turn NoData into 0
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent, extent="NotCaribbean"):
    out_raster = arcpy.sa.Reclassify("Emarine", "Value", "0 1;1 NODATA", "DATA"); out_raster.save("Emarine1")

# Make estuaries hubs

Select all estuaries from the NWI that are deep water, but not the buffer into marine

In [14]:
arcpy.analysis.Select(estuary, "estuary1", "WETLAND_TY = 'Estuarine and Marine Deepwater' And ATTRIBUTE <> 'M1UBL'")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\E_ConMarineResistanceAndHubs.gdb\\estuary1'>

In [15]:
# add and calculate description field to use for raster conversion
arcpy.management.CalculateField("estuary1", "raster", 1, "PYTHON3", '', "SHORT")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\E_ConMarineResistanceAndHubs.gdb\\estuary1'>

In [16]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent, extent="NotCaribbean", mask="Emarine1"):
    arcpy.conversion.PolygonToRaster("estuary1", "raster", "estuary2", "CELL_CENTER", "NONE", Rextent)

## Identify areas in the top 10% of the marine zonation runs

In [17]:
# use a conditional statement to pull out the top 10%, use environemnts to clip to inland
with arcpy.EnvManager(outputCoordinateSystem=sr, mask="Emarine1"):
    out_raster = arcpy.sa.Con(Zonation, 1, '', "VALUE >90"); out_raster.save("Top10")

## combine estuary hubs with zontion hubs

In [18]:
# combine rasters, keeping max value
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent, extent="NotCaribbean", mask="Emarine1"):
    out_raster = arcpy.sa.CellStatistics(["Top10", "estuary2"],"MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("PotHub")

## convert to 90 meter raster and back to vector to make sure it won't throw errors in corridor analysis
Because we need to do corridor analysis at 90 meters, we need to make sure that after cores are converted to 90 meter rasters, there aren't small isolated cores

In [19]:
# resample to 90 meters using default esri method
arcpy.management.Resample("PotHub", "PotHub_90", "90 90", '')

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\E_ConMarineResistanceAndHubs.gdb\\PotHub_90'>

## Calculate size of potential hubs

In [20]:
# use region group to create clumps of isolated potential hubs
out_raster = arcpy.sa.RegionGroup("PotHub_90", "EIGHT", '', '', '0'); out_raster.save("PotHub3a")

In [21]:
# pull out clumps that are larger than 2,000 ha (5,000 acres), keep their region group value
# this is for the 90 meter resolution version
out_raster = arcpy.sa.Con("PotHub3a", "PotHub3a", '', "Count > 2498 And Link = 1"); out_raster.save("PotHub4")

In [22]:
# convert to vector
arcpy.conversion.RasterToPolygon("PotHub4", "PotHub5", "NO_SIMPLIFY", "Value", "MULTIPLE_OUTER_PART", None)

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\E_ConMarineResistanceAndHubs.gdb\\PotHub5'>

In [23]:
# add field that LinkageMapper will recognize
arcpy.management.CalculateField("PotHub5", "UID", "!Id!", "PYTHON3", '', "SHORT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\E_ConMarineResistanceAndHubs.gdb\\PotHub5'>

In [25]:
# copy features
arcpy.management.CopyFeatures("PotHub5", Hubs, '', None, None, None)

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\Ea_ConMarine_LM_Input.gdb\\Hubs'>

# Make resistance raster

In [26]:
# flip zonation results so that high is low and low is high to make resistance layer
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent, extent="NotCaribbean", mask="Emarine1"):
    out_raster = arcpy.sa.Minus(100, Zonation); out_raster.save("ZFlipEMarine")

## bring in potential corridors
This year we are trying a new method where we use corridors from other projects to try to help keep corridors more stable. The idea is that we use our old corridors and corridors from other projects and use them to limit the resistance raster to try to get corridors to move through them.

Rua made the potential corridors for the marine and estuarine environment

The data from the Gulf are cutting out estuarine areas that we don't want cut out. In this raster, the 1s are the potential corridors. The 0s are a mixed class. They include some areas that are NOT corridors, but they also include estuarine areas. I'm trying to fix this by giving all estuarine areas a value of 1

In [27]:
# make a copy of the estuaries for edits
# make a copy of the subregion layer for edits
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent, extent="NotCaribbean"):
    arcpy.management.CopyFeatures(estuary, "estuaryDeep", '', None, None, None)

In [28]:
# add and calculate description field to use for raster conversion
arcpy.management.CalculateField("estuaryDeep", "raster", 1, "PYTHON3", '', "SHORT")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\E_ConMarineResistanceAndHubs.gdb\\estuaryDeep'>

In [29]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent, extent="NotCaribbean"):
    arcpy.conversion.PolygonToRaster("estuaryDeep", "raster", "estuaryDeep2", "CELL_CENTER", "NONE", Rextent)

In [30]:
# The data from the Gulf are cutting out estuarine areas that we don't want cut out. In this raster, the 1s are the potential corridors. The 0s are a mixed class. They include some areas 
# that are NOT corridors, but they also include estuarine areas. I'm trying to fix this by giving all estuarine areas a value of 1
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent, extent="NotCaribbean", mask="Emarine1"):
    out_raster = arcpy.sa.CellStatistics([mask, "estuaryDeep2"], "MAXIMUM", 'DATA'); out_raster.save("mask2")

In [34]:
# reclassify mask to remove zeros
out_raster = arcpy.sa.Reclassify("mask2", "Value", "0 NODATA;1 1", "DATA"); out_raster.save("mask3")

In [35]:
# limit resistance raster to areas around previous corridors, this is the most dramatic option and does not allow
# linkagemapper to even consider areas that were not around previous corridors
out_raster = arcpy.sa.ExtractByMask("ZFlipEMarine", "mask3"); out_raster.save("MResist2")

## resample because 30 meter for the whole SA area is too large for Linkagemapper  to run

In [37]:
# might need to resample to 60 or 90 meter pixels like I had to do last year
# I tried all the options (Nearest,Majority, Bilinear, Cubic) it is a tough call
# the bilinear did a little better job retaining linear features
arcpy.management.Resample("MResist2", Resist, "90 90", "BILINEAR")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\Ea_ConMarine_LM_Input.gdb\\Resistance'>

## This is too big to run at once, need to seperate into pieces

In [38]:
# pull out only the Gulf of Mexico polygon from the subregions
arcpy.analysis.Select(subR, "GulfOfMexico", "SubRgn = 'Gulf of Mexico'")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\E_ConMarineResistanceAndHubs.gdb\\GulfOfMexico'>

In [39]:
# buffer GoM by 200km
arcpy.analysis.Buffer("GulfOfMexico", "GulfOfMexico_200kmBuff", "200 Kilometers", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\E_ConMarineResistanceAndHubs.gdb\\GulfOfMexico_200kmBuff'>

In [41]:
# clip resistance layer to buffered GoM
out_raster = arcpy.sa.ExtractByMask(Resist, "GulfOfMexico_200kmBuff"); out_raster.save(ResistGoM)

In [ ]:
# select hubs that intersect buffered GoM
select = arcpy.management.SelectLayerByLocation(Hubs, "INTERSECT", "GulfOfMexico_200kmBuff", None, "NEW_SELECTION", "NOT_INVERT")
arcpy.conversion.FeatureClassToFeatureClass(select, OutWorkspace2, "HubsGoM") 

In [47]:
# pull out only the Atlantic Marine polygon from the subregions
arcpy.analysis.Select(subR, "Atlantic", "SubRgn = 'Atlantic Marine'")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\E_ConMarineResistanceAndHubs.gdb\\Atlantic'>

In [48]:
# buffer Atlantic by 200km
arcpy.analysis.Buffer("Atlantic", "Atlantic_200kmBuff", "200 Kilometers", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\E_ConMarineResistanceAndHubs.gdb\\Atlantic_200kmBuff'>

In [49]:
# clip resistance layer to buffered Atlantic
out_raster = arcpy.sa.ExtractByMask(Resist, "Atlantic_200kmBuff"); out_raster.save(ResistA)

In [50]:
# select hubs that intersect buffered GoM
select = arcpy.management.SelectLayerByLocation(Hubs, "INTERSECT", "Atlantic_200kmBuff", None, "NEW_SELECTION", "NOT_INVERT")
arcpy.conversion.FeatureClassToFeatureClass(select, OutWorkspace2, "HubsA") 

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\Ea_ConMarine_LM_Input.gdb\\HubsA'>

In [33]:
end = time.time()
print(end - start)

2901.4779844284058
